In [2]:
#Importacion de las librerias necesarias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import mysql.connector
from mysql.connector import Error
import sqlalchemy
from sqlalchemy import create_engine
from fuzzywuzzy import process
import chardet
import os

#RUTA PRINCIPAL DONDE SE ENCUENTRAN LOS ARCHIVOS CSV
ruta = input("Introduzca la ruta donde se encuentran los archivos csv a cargar"+"/")

# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
# :::::::::::::::::::::::::::::::::RUTAS:::::::::::::::::::::::::::::::::
# :::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::

#Creacion de una lista de strings para las los nombres de los archivos csv
lis = [file for file in os.listdir(ruta) if file.endswith(".csv")]


c:\Users\user\AppData\Local\Programs\Python\Python310\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
lis

['Clientes.csv',
 'Compra.csv',
 'Gasto.csv',
 'Localidades.csv',
 'Proveedores.csv',
 'Sucursales.csv',
 'Venta.csv']

In [4]:
rute_lis = [ruta + "/" + file for file in lis]
rute_lis

['C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Clientes.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Compra.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Gasto.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Localidades.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Proveedores.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Sucursales.csv',
 'C:\\Users\\user\\Desktop\\daniel\\DATA SCIENCE\\HENRY\\PROYECTO ETL\\automatic-ETL\\Datasets/Venta.csv']

In [5]:
new_lis = []
for ruta in rute_lis:
    with open(ruta,"rb") as x:
        data = x.read()
        y = chardet.detect(data)
        new_lis.append(y["encoding"])
new_lis


['UTF-8-SIG',
 'ISO-8859-1',
 'ascii',
 'UTF-8-SIG',
 'ISO-8859-1',
 'UTF-8-SIG',
 'ascii']

In [6]:
dflist = [pd.read_csv(ruta, encoding=encod, on_bad_lines='skip',engine="python",sep=None) for ruta, encod in zip(rute_lis,new_lis)]



In [7]:
for df in dflist:
    print(df.shape)
    print(list(df.columns))

(2672, 10)
['ID', 'Provincia', 'Nombre_y_Apellido', 'Domicilio', 'Telefono', 'Edad', 'Localidad', 'X', 'Y', 'col10']
(11539, 9)
['IdCompra', 'Fecha', 'Fecha_AÃ±o', 'Fecha_Mes', 'Fecha_Periodo', 'IdProducto', 'Cantidad', 'Precio', 'IdProveedor']
(8640, 5)
['IdGasto', 'IdSucursal', 'IdTipoGasto', 'Fecha', 'Monto']
(4142, 14)
['categoria', 'centroide_lat', 'centroide_lon', 'departamento_id', 'departamento_nombre', 'fuente', 'id', 'localidad_censal_id', 'localidad_censal_nombre', 'municipio_id', 'municipio_nombre', 'nombre', 'provincia_id', 'provincia_nombre']
(14, 7)
['IDProveedor', 'Nombre', 'Address', 'City', 'State', 'Country', 'departamen']
(31, 7)
['ID', 'Sucursal', 'Direccion', 'Localidad', 'Provincia', 'Latitud', 'Longitud']
(46645, 10)
['IdVenta', 'Fecha', 'Fecha_Entrega', 'IdCanal', 'IdCliente', 'IdSucursal', 'IdEmpleado', 'IdProducto', 'Precio', 'Cantidad']


In [8]:
for df in dflist:
    if "Domicilio" and "X" and "Y" and "Telefono" in list(df.columns):
        clientesdf = df
    if "IdCompra" in list(df.columns) and df.shape[1] == 9:
        compradf = df
    if "IdGasto" in list(df.columns) and df.shape[1] == 5:
        gastodf = df
    if "centroide_lat" and "centroide_lon" and "localidad_censal_nombre" in list(df.columns):
        localidadesdf = df
    if "IDProveedor" and "Nombre" and "Address" and "City" in list(df.columns):
        proveedoresdf = df
    if "Sucursal" and "Direccion" and "Localidad" in list(df.columns):
        sucursalesdf = df
    if "IdVenta" in list(df.columns) and df.shape[1] == 10:
        ventadf = df

In [9]:
sucursalesdf

,ID,Sucursal,Direccion,Localidad,Provincia,Latitud,Longitud
0,1,Cabildo,Av. Cabildo 1342,Ciudad de Buenos Aires,Ciudad de Buenos Aires,"-34,5678060","-58,4495720"
1,2,Palermo 1,Guatemala 5701,CABA,CABA,"-34,5790350","-58,4335660"
2,3,Palermo 2,Gral. Lucio Norberto Mansilla 2668,CABA,C deBuenos Aires,"-34,5959660","-58,4051500"
3,4,Corrientes,Av. Corrientes 2352,Ciudad de Buenos Aires,Bs As,"-34,6046850","-58,3987640"
4,5,Almagro,Venezuela 3650,Capital,Bs.As.,"-34,6173080","-58,4161790"
5,6,Caballito,Av. Rivadavia 4708,Capital Federal,Ciudad de Buenos Aires,"-34,6163030","-58,4318490"
6,7,Flores,Av. Rivadavia 5746,CapFed,Buenos Aires,"-34,6228680","-58,4464490"
7,8,Alberdi,Av. Juan Bautista Alberdi 1634,Cap. Fed.,Buenos Aires,"-34,6273060","-58,4514980"
8,9,Deposito,Pedernera 530,Cap. Federal,Buenos Aires,"-34,6351340","-58,4615130"
9,10,Velez,Av. Juan Bautista Justo 7738,Cdad de Buenos Aires,Ciudad de Buenos Aires,"-34,6312570","-58,4980630"


In [10]:
compradf

,IdCompra,Fecha,Fecha_AÃ±o,Fecha_Mes,Fecha_Periodo,IdProducto,Cantidad,Precio,IdProveedor
0,1,1/30/2015,2015,1,201501,42832,13,560.51,12
1,2,1/30/2015,2015,1,201501,42833,11,497.58,7
2,3,1/30/2015,2015,1,201501,42834,1,588.50,6
3,4,1/30/2015,2015,1,201501,42835,9,567.66,14
4,5,1/30/2015,2015,1,201501,42839,14,231.31,2
...,...,...,...,...,...,...,...,...,...
11534,11535,12/25/2020,2020,12,202012,42978,6,88.29,10
11535,11536,12/25/2020,2020,12,202012,42979,2,81.24,10
11536,11537,12/25/2020,2020,12,202012,42980,1,71.67,3
11537,11538,12/25/2020,2020,12,202012,42981,3,35.55,9


In [11]:
sucursalesdf.Provincia.replace({"CABA":"Buenos Aires","Prov de Bs As.":"Buenos Aires","Pcia Bs AS":"Buenos Aires"},inplace=True)
provincia_sucursalesdf = sucursalesdf.Provincia.unique()
provincia_mstr = localidadesdf.provincia_nombre.unique()
normalized = []
def get_matches(query,choices):
    for i in query:
        tuple = process.extractOne(i,choices)
        normalized.append(tuple[0])
    return normalized
provincia_normalizada = get_matches(provincia_sucursalesdf,provincia_mstr)

In [12]:
provincia_normalizada

['Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Santa Fe',
 'Córdoba',
 'Córdoba',
 'Tucumán',
 'Mendoza',
 'Río Negro']

In [13]:
for df in dflist:
    print(len(df.columns))

10
9
5
14
7
7
10
